<a href="https://colab.research.google.com/github/blenature/8085simulator.github.io/blob/master/Image%20processing%20with%20homography%20and%20transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np

# Load the image
img = cv2.imread('granma.jpg')

In [ ]:
# Display the image and ask the user to click on the four corners
cv2.imshow('Image', img)
corners = []


In [ ]:
def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONUP:
        corners.append((x, y))
        cv2.circle(img, (x, y), 5, (0, 0, 255), -1)
        cv2.imshow('Image', img)

cv2.setMouseCallback('Image', mouse_callback)

cv2.waitKey(0)

In [ ]:
# Calculate the projective transformation matrix
src_pts = np.float32(corners)
dst_pts = np.float32([[0, 0], [1000, 0], [1000, 1000], [0, 1000]])

In [ ]:
# Calculate the homography matrix using source and destination points
A = []
for i in range(4):
    x, y = src_pts[i]
    u, v = dst_pts[i]
    A.append([x, y, 1, 0, 0, 0, -u*x, -u*y, -u])
    A.append([0, 0, 0, x, y, 1, -v*x, -v*y, -v])

In [ ]:
A = np.asarray(A)
U, S, Vh = np.linalg.svd(A)
L = Vh[-1,:] / Vh[-1,-1]
H = L.reshape(3, 3)

In [ ]:
# Apply the transformation and create a new image of size 1000 × 1000
h, w, _ = img.shape
transformed = np.zeros((1000, 1000, 3), dtype=np.uint8)
for y in range(1000):
    for x in range(1000):
        p = np.dot(np.linalg.inv(H), [x, y, 1])
        p = p / p[-1]
        u, v = p[:-1]
        if 0 <= u < w and 0 <= v < h:
            transformed[y, x] = img[int(v), int(u)]

In [ ]:
# Transpose the image
transformed = np.transpose(transformed, (1, 0, 2))

In [ ]:
# Display the final image
cv2.imshow('Result', transformed)
cv2.waitKey(0)
cv2.destroyAllWindows()